In [ ]:
import sys 
sys.path.append('../src') 

import numpy as np
import matplotlib.pyplot as plt
# import torch

import KPCA
from utils import get_data, calc_energy

import pysindy as ps

In [ ]:
dir_path='../../npz_data/dataset_cavity'
in_filename = f"cavitytransU80_data_Re{Re:g}_Wi{Wi:g}_beta{beta:g}.npz" # u,v,B
fields = np.load(f'{dir_path}/{in_filename}', allow_pickle=True)["fields"].item()

In [ ]:
fields.keys()

dict_keys(['vel-u', 'vel-v', 'Bxx', 'Bxy', 'Byy', 'traceA'])

In [ ]:
# Re = 10
# beta = .5
# Wi = 5
# eps = 1
# case = 'cavity'
# theta = (1.0-beta)/(Re*Wi)
# X, X_mean = get_data(Re, Wi, beta, case='cavity_ref', n_data=-2, dir_path='../../npz_data/dataset_cavity')


Re = 0.01
beta = .5
Wi = 5
eps = 1
case = 'cavity'
theta = (1.0-beta)/(Re*Wi)
X, X_mean = get_data(Re, Wi, beta, case='cavity', n_data=99, dir_path='../../npz_data')


print(theta)

10.0


In [ ]:
# get modes
kernel_linear = KPCA.KernelPCA()
kernel_oldroyd = KPCA.KernelPCA()


kernel_linear.fit(X.T, n_components=2, kernel='linear', theta = theta,degree=1)
Phi_lin = kernel_linear.transform(X.T, theta)

kernel_oldroyd.fit(X.T, n_components=2, kernel='oldroyd', theta = theta,degree=1)
Phi_old = kernel_oldroyd.transform(X.T, theta)

In [ ]:
nu = 1
lam = 0.1
dg = 3
bias = False

model_lin = ps.SINDy(
    optimizer=ps.SR3(relax_coeff_nu=nu, reg_weight_lam=lam, max_iter=2000),
    feature_library=ps.PolynomialLibrary(degree=dg, include_bias=bias)
)

model_old = ps.SINDy(
    optimizer=ps.SR3(relax_coeff_nu=nu, reg_weight_lam=lam, max_iter=2000),
    feature_library=ps.PolynomialLibrary(degree=dg, include_bias=bias)
)

model_lin.fit(Phi_lin,t=1e-2)
model_lin.print()

print('=========')

model_lin.fit(Phi_old,t=1e-2)
model_lin.print()


(x0)' = -0.891 x0 + -49.303 x1 + -1.278 x0 x1 + -0.493 x1^2
(x1)' = 31.334 x0 + -1.346 x1 + -0.798 x0 x1
(x0)' = -9.944 x0 + 108.942 x1 + -2.839 x0 x1 + 1.435 x1^2
(x1)' = 12.786 x0 + 16.776 x1 + 2.787 x0 x1 + 2.025 x1^2


(0.01, -1e-05)